<a href="https://colab.research.google.com/github/lrezqi/ICKD-DCKD/blob/main/ImageNet/demo/cifar_kd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Distilling knowledge in models pretrained on CIFAR-10/100 datasets, using ***torchdistill***

## 1. Make sure you have access to GPU/TPU
Google Colab: *Runtime* -> *Change runtime type* -> *Hardware accelarator*: "GPU" or "TPU"

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Is GPU available:", tf.config.list_physical_devices('GPU'))


TensorFlow version: 2.19.0
Is GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:

if tf.config.list_physical_devices('GPU'):
    with tf.device('/GPU:0'):
        # Run a simple operation on the GPU
        a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
        b = tf.constant([[5.0, 6.0], [7.0, 8.0]])
        c = tf.matmul(a, b)
        print("GPU operation result:", c)
else:
    print("No GPU available for TensorFlow.")


GPU operation result: tf.Tensor(
[[19. 22.]
 [43. 50.]], shape=(2, 2), dtype=float32)


In [3]:
!nvidia-smi

Tue Aug 26 21:05:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P0             27W /   70W |     116MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
%cd /content
!git clone https://github.com/lrezqi/ICKD-DCKD.git


/content
Cloning into 'ICKD-DCKD'...
remote: Enumerating objects: 610, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 610 (delta 45), reused 0 (delta 0), pack-reused 540 (from 3)
Receiving objects: 100% (610/610), 836.12 KiB | 9.84 MiB/s, done.
Resolving deltas: 100% (292/292), done.


In [43]:
!pip install pudb
!pip install tensorboard_logger


In [6]:
file_path = '/content/ICKD-DCKD/Cifar100/helper/util.py'

with open(file_path, 'r') as f:
    lines = f.readlines()

with open(file_path, 'w') as f:
    for line in lines:
        f.write(line.replace('view(-1)', 'reshape(-1)') if 'view(-1)' in line else line)

print("✅ Remplacement de .view(-1) par .reshape(-1) effectué.")


✅ Remplacement de .view(-1) par .reshape(-1) effectué.


In [9]:
with open(file_path, 'w') as f:
    in_block = False
    for line in lines:
        if 'raise NotImplementedError(opt.distill)' in line and not in_block:
            f.write("    elif opt.distill == 'dckd':\n")
            f.write("        model_s.train()\n")
            f.write("        model_t.eval()\n")
            f.write("        for batch_idx, (input, target, _, index) in enumerate(train_loader):\n")
            f.write("            input = input.cuda()\n")
            f.write("            target = target.cuda()\n")
            f.write("            feat_s, logit_s = model_s(input, is_feat=True)\n")
            f.write("            with torch.no_grad():\n")
            f.write("                feat_t, logit_t = model_t(input, is_feat=True)\n")
            f.write("            loss_cls = criterion_cls(logit_s, target)\n")
            f.write("            loss_div = criterion_div(logit_s, logit_t)\n")
            f.write("            loss_kd = criterion_kd(feat_s, feat_t)\n")
            f.write("            loss = opt.gamma * loss_cls + opt.alpha * loss_div + opt.beta * loss_kd\n")
            f.write("            optimizer.zero_grad()\n")
            f.write("            loss.backward()\n")
            f.write("            optimizer.step()\n")
            in_block = True
        else:
            f.write(line)

print("✅ Bloc 'dckd' inséré avec succès dans loops.py")

NameError: name 'file_path' is not defined

In [44]:
# Dans une cellule Colab
%cd ICKD-DCKD/Cifar100



/content/ICKD-DCKD/Cifar100/ICKD-DCKD/Cifar100


In [9]:


# 2. Vérifier CUDA
import torch
print(f"CUDA disponible: {torch.cuda.is_available()}")
print(f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

# 3. Test import DCKD
%cd /content/ICKD-DCKD/Cifar100
from distiller_zoo.DCKD import DCKDLoss
print("✅ DCKD importé avec succès!")


CUDA disponible: True
Device: Tesla T4
/content/ICKD-DCKD/Cifar100
✅ DCKD importé avec succès!


In [10]:
# Test DCKD
!python train_student.py \
  --path_t ./save/models/resnet32x4_vanilla/ckpt_epoch_240.pth \
  --distill dckd \
  --model_s resnet8x4 \
  -a 0.5 -b 2.5 \
  --trial test \
  --epochs 5 \
  --batch_size 32  # Plus petit pour Colab

2025-08-26 21:18:06.284487: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756243086.306367   14546 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756243086.313104   14546 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756243086.329791   14546 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756243086.329826   14546 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756243086.329830   14546 computation_placer.cc:177] computation placer alr

In [22]:

!ls ICKD-DCKD


Cifar100  Dockerfile  ImageNet	README.md  Segmentation


In [27]:
!ls ICKD-DCKD/Cifar100/models/


classifier.py  mobilenetv2.py  resnet.py    ShuffleNetv1.py  util.py  wrn.py
__init__.py    __pycache__     resnetv2.py  ShuffleNetv2.py  vgg.py


In [28]:
%cd ICKD-DCKD/Cifar100
!mkdir -p checkpoints


/content/ICKD-DCKD/Cifar100


In [30]:
# Installer gdown si nécessaire
!pip install gdown

# Télécharger le checkpoint dans checkpoints/
!gdown --id FILE_ID --output checkpoints/wrn_40_2_best.pth


/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=FILE_ID

but Gdown can't. Please check connections and permissions.


In [21]:
# Exemple : remplace URL par ton lien de téléchargement direct
!wget -O checkpoints/wrn_16_2_best.pth "https://drive.google.com/drive/u/0/home"


checkpoints/wrn_16_2_best.pth: No such file or directory


In [32]:
# Exemple : remplace URL par ton lien de téléchargement direct
!wget -O checkpoints/wrn_40_2_best.pth "https://drive.google.com/drive/u/0/home"


--2025-08-26 22:27:12--  https://drive.google.com/drive/u/0/home
Resolving drive.google.com (drive.google.com)... 173.194.212.101, 173.194.212.113, 173.194.212.139, ...
Connecting to drive.google.com (drive.google.com)|173.194.212.101|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://accounts.google.com/ServiceLogin?service=wise&passive=1209600&osid=1&continue=https://drive.google.com/drive/u/0/home&followup=https://drive.google.com/drive/u/0/home [following]
--2025-08-26 22:27:12--  https://accounts.google.com/ServiceLogin?service=wise&passive=1209600&osid=1&continue=https://drive.google.com/drive/u/0/home&followup=https://drive.google.com/drive/u/0/home
Resolving accounts.google.com (accounts.google.com)... 108.177.12.84, 2607:f8b0:400c:c11::54
Connecting to accounts.google.com (accounts.google.com)|108.177.12.84|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://accounts.google.com/InteractiveLogin?continue=htt

In [33]:
!python train_student.py \
  --path_t checkpoints/wrn_40_2_best.pth \
  --model_t wrn_40_2 \
  --distill dckd \
  --model_s wrn_16_2 \
  -a 0.5 -b 2.5 \
  --trial test \
  --epochs 5 \
  --batch_size 64


2025-08-26 22:29:41.166878: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756247381.188311   32148 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756247381.194887   32148 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756247381.211887   32148 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756247381.211914   32148 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756247381.211918   32148 computation_placer.cc:177] computation placer alr

In [34]:
!head -n 5 checkpoints/wrn_40_2_best.pth
!wc -c checkpoints/wrn_40_2_best.pth


<!doctype html><html lang="en-US" dir="ltr"><head><base href="https://accounts.google.com/v3/signin/"><link rel="preconnect" href="//www.gstatic.com"><meta name="referrer" content="origin"><style data-href="https://www.gstatic.com/_/mss/boq-identity/_/ss/k=boq-identity.AccountsSignInUi.R2Hzgy-vPK4.L.W.O/am=gQMCs2WMrQ0AlP8_IQBFAIAAAAUAAAAAAAAAgMAAAEBICAAE/d=1/ed=1/rs=AOaEmlExhiEzYUMtISVTz_K3UEzDXn0j2Q/m=identifierview,_b,_tp" nonce="5FOzpuO4UT6Km47-nC9VHQ">@-webkit-keyframes quantumWizBoxInkSpread{0%{-webkit-transform:translate(-50%,-50%) scale(0.2);-webkit-transform:translate(-50%,-50%) scale(0.2);transform:translate(-50%,-50%) scale(0.2)}to{-webkit-transform:translate(-50%,-50%) scale(2.2);-webkit-transform:translate(-50%,-50%) scale(2.2);transform:translate(-50%,-50%) scale(2.2)}}@keyframes quantumWizBoxInkSpread{0%{-webkit-transform:translate(-50%,-50%) scale(0.2);-webkit-transform:translate(-50%,-50%) scale(0.2);transform:translate(-50%,-50%) scale(0.2)}to{-webkit-transform:transla

In [35]:
# Installe gdown si nécessaire
!pip install gdown

# Télécharge le checkpoint WRN-40-2 depuis Google Drive
!gdown --id TON_FILE_ID --output checkpoints/wrn_40_2_best.pth


/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=TON_FILE_ID

but Gdown can't. Please check connections and permissions.


In [1]:
!ls checkpoints
# wrn_40_2_best.pth


ls: cannot access 'checkpoints': No such file or directory


In [26]:
!ls models


ls: cannot access 'models': No such file or directory


In [16]:

!git commit -m "Correction des bugs de DCKD et nettoyage"

!git push origin main


!git add Cifar100/save



fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [4]:
%cd /content


/content


In [10]:
!python3 train_teacher.py \
  --model wrn_40_2 \
  --dataset cifar100 \
  --epochs 200 \
  --batch_size 128 \
  --learning_rate 0.1 \
  --momentum 0.9 \
  --weight_decay 5e-4 \
  --save_dir checkpoints


2025-08-27 00:14:09.569169: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756253649.589183   10664 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756253649.595448   10664 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756253649.611279   10664 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756253649.611303   10664 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756253649.611307   10664 computation_placer.cc:177] computation placer alr

In [11]:
%cd /content/ICKD-DCKD/Cifar100

!python3 train_teacher.py \
  --model wrn_40_2 \
  --dataset cifar100 \
  --epochs 200 \
  --batch_size 128 \
  --learning_rate 0.1 \
  --momentum 0.9 \
  --weight_decay 5e-4 \
  --save-path checkpoints/wrn_40_2_best.pth


/content/ICKD-DCKD/Cifar100
2025-08-27 00:24:05.708669: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756254245.741340   13097 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756254245.751163   13097 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756254245.778357   13097 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756254245.780150   13097 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756254245.780171   13097 computation_placer.cc

In [12]:
!python3 train_teacher.py -h


2025-08-27 00:26:03.510049: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756254363.529311   13589 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756254363.535121   13589 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756254363.551698   13589 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756254363.551725   13589 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756254363.551731   13589 computation_placer.cc:177] computation placer alr

In [15]:
!python3 train_teacher.py \
  --model wrn_40_2 \
  --dataset cifar100 \
  --epochs 200 \
  --batch_size 128 \
  --learning_rate 0.1 \
  --momentum 0.9 \
  --weight_decay 5e-4 \
  --save_freq 50 \
  --lr_decay_epochs 60,120,160 \
  --trial teacher_run


2025-08-27 00:33:16.377922: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756254796.398388   15397 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756254796.404765   15397 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756254796.420210   15397 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756254796.420236   15397 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756254796.420239   15397 computation_placer.cc:177] computation placer alr

In [18]:
%cd /content/ICKD-DCKD/Cifar100

!python3 train_teacher.py \
  --model wrn_40_2 \
  --dataset cifar100 \
  --epochs 60 \
  --batch_size 128 \
  --learning_rate 0.1 \
  --momentum 0.9 \
  --weight_decay 5e-4 \
  --save_freq 50 \
  --lr_decay_epochs 60,120,160 \
  --trial 1


/content/ICKD-DCKD/Cifar100
2025-08-27 01:52:14.307723: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756259534.327692   40552 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756259534.333886   40552 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756259534.349359   40552 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756259534.349383   40552 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756259534.349387   40552 computation_placer.cc

In [19]:
!python3 train_student.py \
  --model_t wrn_40_2 \
  --path_t checkpoint/cifar100/wrn_40_2/1/epoch_200.pth \
  --model_s wrn_16_2 \
  --distill dckd \
  --batch_size 128 \
  --epochs 60 \
  --lr 0.1 \
  --momentum 0.9 \
  --weight_decay 5e-4 \
  --trial 1


2025-08-27 02:37:44.414784: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756262264.436818   54621 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756262264.442995   54621 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756262264.459811   54621 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756262264.459836   54621 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756262264.459840   54621 computation_placer.cc:177] computation placer alr

In [28]:
!ls -d checkpoint*



ls: cannot access 'checkpoint*': No such file or directory


In [29]:
!ls checkpoint*/


ls: cannot access 'checkpoint*/': No such file or directory


In [34]:
%cd /content/ICKD-DCKD/Cifar100

!python3 train_student.py \
  --model_t wrn_40_2 \
  --path_t save/models/wrn_40_2_cifar100_lr_0.1_decay_0.0005_trial_1/wrn_40_2_best.pth \
  --model_s wrn_16_2 \
  --distill dckd \
  --batch_size 128 \
  --epochs 60 \
  --learning_rate 0.1 \
  --momentum 0.9 \
  --weight_decay 5e-4 \
  --trial 1 \
  --save_freq 20


/content/ICKD-DCKD/Cifar100
2025-08-27 02:52:01.746245: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756263121.777783   58210 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756263121.787655   58210 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756263121.814772   58210 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756263121.814809   58210 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756263121.814817   58210 computation_placer.cc

In [36]:
!python3 train_student.py -h


2025-08-27 02:54:42.699546: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756263282.719252   58953 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756263282.725348   58953 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756263282.740833   58953 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756263282.740860   58953 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756263282.740864   58953 computation_placer.cc:177] computation placer alr

In [37]:
%cd /content/ICKD-DCKD/Cifar100

!python3 train_student.py \
  --model_t wrn_40_2 \
  --path_t save/models/wrn_40_2_cifar100_lr_0.1_decay_0.0005_trial_1/wrn_40_2_best.pth \
  --model_s wrn_16_2 \
  --distill dckd \
  --alpha 0.5 \
  --beta 2.5 \
  --tau 4.0 \
  --batch_size 128 \
  --epochs 60 \
  --learning_rate 0.1 \
  --momentum 0.9 \
  --weight_decay 5e-4 \
  --trial 1 \
  --save_freq 20


/content/ICKD-DCKD/Cifar100
2025-08-27 02:56:00.076606: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756263360.096602   59285 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756263360.102645   59285 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756263360.117974   59285 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756263360.117999   59285 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756263360.118003   59285 computation_placer.cc

In [38]:
!python3 train_student.py -h


2025-08-27 02:59:53.469678: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756263593.501854   60257 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756263593.511949   60257 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756263593.536126   60257 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756263593.536165   60257 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756263593.536173   60257 computation_placer.cc:177] computation placer alr

In [39]:
%cd /content/ICKD-DCKD/Cifar100

!python3 train_student.py \
  --model_t wrn_40_2 \
  --path_t save/models/wrn_40_2_cifar100_lr_0.1_decay_0.0005_trial_1/wrn_40_2_best.pth \
  --model_s wrn_16_2 \
  --distill dckd \
  --gamma 1.0 \
  --alpha 0.5 \
  --beta 2.5 \
  --kd_T 4.0 \
  --batch_size 128 \
  --epochs 60 \
  --learning_rate 0.1 \
  --momentum 0.9 \
  --weight_decay 5e-4 \
  --trial 1 \
  --save_freq 20


/content/ICKD-DCKD/Cifar100
2025-08-27 03:03:20.555032: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756263800.576644   61117 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756263800.584501   61117 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756263800.610268   61117 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756263800.610299   61117 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756263800.610307   61117 computation_placer.cc

In [42]:
%cd /content/ICKD-DCKD/Cifar100

!python3 train_student.py \
  --model_t wrn_40_2 \
  --path_t save/models/wrn_40_2_cifar100_lr_0.1_decay_0.0005_trial_1/wrn_40_2_best.pth \
  --model_s wrn_16_2 \
  --distill dckd \
  --gamma 1.0 \
  --alpha 0.5 \
  --beta 2.5 \
  --kd_T 4.0 \
  --batch_size 128 \
  --epochs 60 \
  --learning_rate 0.1 \
  --momentum 0.9 \
  --weight_decay 5e-4 \
  --trial 1 \
  --save_freq 20


/content/ICKD-DCKD/Cifar100
2025-08-27 03:13:12.934483: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756264392.954957   63643 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756264392.961110   63643 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756264392.976945   63643 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756264392.976970   63643 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756264392.976975   63643 computation_placer.cc

In [45]:


!git clone https://github.com/lrezqi/ICKD-DCKD.git


Cloning into 'ICKD-DCKD'...
remote: Enumerating objects: 615, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 615 (delta 46), reused 0 (delta 0), pack-reused 540 (from 3)
Receiving objects: 100% (615/615), 840.38 KiB | 2.40 MiB/s, done.
Resolving deltas: 100% (293/293), done.


In [49]:



!git push origin main






fatal: could not read Username for 'https://github.com': No such device or address


In [35]:
!pip install pudb
!pip install tensorboard_logger

In [33]:
%cd /content/ICKD-DCKD/Cifar100
!find . -type f -name "*.pth"



/content/ICKD-DCKD/Cifar100
./save/models/wrn_40_2_cifar100_lr_0.1_decay_0.0005_trial_1/ckpt_epoch_50.pth
./save/models/wrn_40_2_cifar100_lr_0.1_decay_0.0005_trial_1/wrn_40_2_best.pth
./save/models/wrn_40_2_cifar100_lr_0.1_decay_0.0005_trial_1/wrn_40_2_last.pth
./save/models/wrn_40_2_cifar100_lr_0.1_decay_0.0005_trial_1/ckpt_epoch_100.pth


In [30]:
!ls -d checkpoints


ls: cannot access 'checkpoints': No such file or directory


In [20]:
%cd /content/ICKD-DCKD/Cifar100

!python3 train_student.py \
  --model_t wrn_40_2 \
  --path_t checkpoint/cifar100/wrn_40_2/1/epoch_200.pth \
  --model_s wrn_16_2 \
  --distill dckd \
  --batch_size 128 \
  --epochs 100 \
  --learning_rate 0.1 \
  --momentum 0.9 \
  --weight_decay 5e-4 \
  --trial 1


/content/ICKD-DCKD/Cifar100
2025-08-27 02:39:18.386856: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756262358.406983   55015 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756262358.413069   55015 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756262358.428642   55015 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756262358.428669   55015 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756262358.428674   55015 computation_placer.cc

In [13]:
%cd /content/ICKD-DCKD/Cifar100


/content/ICKD-DCKD/Cifar100


In [6]:
%cd /content/ICKD-DCKD/Cifar100


/content/ICKD-DCKD/Cifar100


In [27]:
!git clone https://github.com/lrezqi/ICKD-DCKD.git


Cloning into 'ICKD-DCKD'...
remote: Enumerating objects: 610, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 610 (delta 45), reused 0 (delta 0), pack-reused 540 (from 3)
Receiving objects: 100% (610/610), 836.12 KiB | 2.67 MiB/s, done.
Resolving deltas: 100% (292/292), done.


## 2. Install dependencies and ***torchdistill***
As of January 6, 2021, it seems that Google Colab requires us to use **CUDA ver. 10.1** for PyTorch.
Thus, install `torch` and `torchvision` with +cu101

In [50]:
%cd /content/ICKD-DCKD/Cifar100
!python3 train_student.py \
  --model_t wrn_40_2 \
  --path_t save/models/wrn_40_2_cifar100_lr_0.1_decay_0.0005_trial_1/wrn_40_2_best.pth \
  --model_s wrn_16_2 \
  --distill dckd \
  --gamma 1.0 \
  --alpha 0.5 \
  --beta 2.5 \
  --kd_T 4.0 \
  --batch_size 128 \
  --epochs 60 \
  --learning_rate 0.1 \
  --momentum 0.9 \
  --weight_decay 5e-4 \
  --trial 1 \
  --save_freq 20


/content/ICKD-DCKD/Cifar100
2025-08-27 03:19:52.284445: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756264792.304294   65408 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756264792.310453   65408 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756264792.326068   65408 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756264792.326091   65408 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756264792.326095   65408 computation_placer.cc

In [ ]:
!pip install pyyaml --upgrade
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torchdistill

     |████████████████████████████████| 276kB 15.1MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=db69fac543e6e884645da5e37d4361779c9bf2296573b6ce02686515efe6d3f2
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4MB 24kB/s 
     |████████████████████████████████| 12.8MB 213kB/s 
     |████████████████████████████████| 7.6MB 7.2MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101
     |██████

## 3. Clone ***torchdistill*** repository to use its example code and configuration files

In [ ]:
!git clone https://github.com/yoshitomo-matsubara/torchdistill.git

Cloning into 'torchdistill'...
remote: Enumerating objects: 447, done.
remote: Counting objects: 100% (447/447), done.
remote: Compressing objects: 100% (224/224), done.
remote: Total 4091 (delta 294), reused 341 (delta 212), pack-reused 3644
Receiving objects: 100% (4091/4091), 822.43 KiB | 19.58 MiB/s, done.
Resolving deltas: 100% (2518/2518), done.


## 4. Distill knowledge in models pretrained on CIFAR-10

Note that the hyperparameters of ResNet, WRN (Wide ResNet), and DenseNet-BC were chosen based on either train/val (splitting 50k samples into train:val = 45k:5k) or cross-validation, according to the original papers.  
For the final run (once the hyperparameters are finalized), the authors used all the training images (50k samples).  
- ResNet: https://github.com/facebookarchive/fb.resnet.torch
- WRN (Wide ResNet): https://github.com/szagoruyko/wide-residual-networks
- DenseNet-BC: https://github.com/liuzhuang13/DenseNet

The following examples demonstrate how to 1) tune hyperparameter and 2) do final-run with ResNet-20 on CIFAR-10 dataset, respectively.

### 4.1 Hyperparameter tuning based on train:val = 45k:5k
Let's start with a small **student model**, ResNet-20, with a pretrained DenseNet-BC (k=12, depth=100) as a **teacher model** for tutorial.  

Open `torchdistill/configs/sample/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-hyperparameter_tuning.yaml` and update hyperparameters as you wish e.g., number of epochs (*num_epochs*), batch size (*batch_size* in *train_data_loader* entry), learning rate (*lr* within *optimizer* entry), and so on.  
By default, the hyperparameters in the example config are identical to those in the final run config.
  
You will find a lot of module names from [PyTorch documentation](https://pytorch.org/docs/stable/index.html) and [torchvision](https://pytorch.org/docs/stable/torchvision/) such as [`SGD`](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD), [`MultiStepLR`](https://pytorch.org/docs/stable/optim.html#torch.optim.lr_scheduler.MultiStepLR), [`CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss), [`CIFAR10`](https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.CIFAR10), [`RandomCrop`](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.RandomCrop) (, and more). You can update their parameters or replace such modules with other modules in the packages. For instance, `SGD` could be replaced with [`Adam`](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam), and then you will change the parameters under `params` (at least delete `momentum` entry as the parameter is not for `Adam`).

In [ ]:
!python torchdistill/examples/image_classification.py --config torchdistill/configs/sample/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-hyperparameter_tuning.yaml --log log/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-hyperparameter_tuning.log

2021/01/10 20:29:25	INFO	torchdistill.common.main_util	Not using distributed mode
2021/01/10 20:29:25	INFO	__main__	Namespace(adjust_lr=False, config='torchdistill/configs/sample/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-hyperparameter_tuning.yaml', device='cuda', dist_url='env://', log='log/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-hyperparameter_tuning.log', start_epoch=0, student_only=False, sync_bn=False, test_only=False, world_size=1)
2021/01/10 20:29:25	INFO	torchdistill.datasets.util	Loading dummy data
170500096it [00:01, 99619332.72it/s]                   
Extracting ./resource/dataset/cifar10/cifar-10-python.tar.gz to ./resource/dataset/cifar10
2021/01/10 20:29:30	INFO	torchdistill.datasets.util	Splitting `dummy` dataset (50000 samples in total)
2021/01/10 20:29:30	INFO	torchdistill.datasets.util	new dataset_id: `cifar10/train` (45000 samples)
2021/01/10 20:29:30	INFO	torchdistill.datasets.util	new dataset_id: `cifar10/val` (5000 samples)
2021/01/10 20:29:30	IN

### 4.2 Final run with hyperparameters determinded by the above hyperparameter-tuning
Once you tune the hyperparameters, you can update the values in **a config file whose name ends with "-final_run.yaml"**. Notice that the only difference between default example configs for hyperparameter tuning and final run is datasets entry.

In [ ]:
!python torchdistill/examples/image_classification.py --config torchdistill/configs/sample/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-final_run.yaml --log log/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-final_run.log

2021/01/10 22:47:35	INFO	torchdistill.common.main_util	Not using distributed mode
2021/01/10 22:47:35	INFO	__main__	Namespace(adjust_lr=False, config='torchdistill/configs/sample/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-final_run.yaml', device='cuda', dist_url='env://', log='log/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-final_run.log', start_epoch=0, student_only=False, sync_bn=False, test_only=False, world_size=1)
2021/01/10 22:47:35	INFO	torchdistill.datasets.util	Loading train data
Files already downloaded and verified
2021/01/10 22:47:36	INFO	torchdistill.datasets.util	0.9210293292999268 sec
2021/01/10 22:47:36	INFO	torchdistill.datasets.util	Loading val data
Files already downloaded and verified
2021/01/10 22:47:36	INFO	torchdistill.datasets.util	0.7336370944976807 sec
2021/01/10 22:47:36	INFO	torchdistill.datasets.util	Loading test data
Files already downloaded and verified
2021/01/10 22:47:37	INFO	torchdistill.datasets.util	0.7437591552734375 sec
2021/01/10 22:4

## 5. More sample configurations, models, datasets...
For CIFAR-10/100 datasets, you can find more [sample configurations](https://github.com/yoshitomo-matsubara/torchdistill/tree/master/configs/sample/) and [models](https://github.com/yoshitomo-matsubara/torchdistill/tree/master/torchdistill/models/classification) in the [***torchdistill***](https://github.com/yoshitomo-matsubara/torchdistill) repository.  
If you would like to use larger datasets e.g., **ImageNet** and **COCO** datasets and models in `torchvision` (or your own modules), refer to the [official configurations](https://github.com/yoshitomo-matsubara/torchdistill/tree/master/configs/official) used in some published papers.  
Experiments with such large datasets and models will require you to use your own machine due to limited disk space and session time (12 hours for free version and 24 hours for Colab Pro) on Google Colab.


# Colab examples for training student models without teacher models
You can find Colab examples for training models without teachers in the [***torchdistill***](https://github.com/yoshitomo-matsubara/torchdistill) repository.